# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821



## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
try :
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity with replication = {'class':'SimpleStrategy','replication_factor':2}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try :
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
create_music_session_table = "CREATE TABLE IF NOT EXISTS MUSIC_SESSIONS"
create_music_session_table = create_music_session_table + "(artist text,item_in_session int, song_length float,session_id int,song_title text, PRIMARY KEY(session_id,item_in_session))"
try :
    session.execute(create_music_session_table)
except Exception as e:
    print(e)

In [10]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO MUSIC_SESSIONS(artist,item_in_session,song_length,session_id,song_title)"
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        session.execute(query, (line[0],int(line[3]),float(line[5]),int(line[8]),line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
query = "SELECT artist,item_in_session,session_id, song_title,song_length FROM MUSIC_SESSIONS WHERE session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist,row.item_in_session,row.song_length,row.session_id,row.song_title)

Faithless 4 495.30731201171875 338 Music Matters (Mark Knight Dub)


In [12]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
create_user_session_table = "CREATE TABLE IF NOT EXISTS USER_SESSIONS"
create_user_session_table = create_user_session_table + "(user_id int, artist text,first_name text,last_name text,song_title text,item_in_session int,session_id int, PRIMARY KEY((user_id,session_id),item_in_session))"
try :
    session.execute(create_user_session_table)
except Exception as e:
    print(e)

# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO USER_SESSIONS(user_id,artist,first_name,last_name,song_title,item_in_session,session_id)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]),line[0],line[1],line[4],line[9],int(line[3]),int(line[8])))

In [16]:
query = "SELECT session_id,user_id,item_in_session,artist,song_title,first_name,last_name FROM USER_SESSIONS WHERE user_id = 8 and session_id = 139"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_id,row.artist,row.first_name,row.last_name,row.song_title,row.item_in_session,row.session_id)

8 Des'ree Kaylee Summers You Gotta Be 1 139
8 Mr Oizo Kaylee Summers Flat 55 3 139
8 Tamba Trio Kaylee Summers Quem Quiser Encontrar O Amor 4 139
8 The Mars Volta Kaylee Summers Eriatarka 5 139
8 Infected Mushroom Kaylee Summers Becoming Insane 6 139
8 Blue October / Imogen Heap Kaylee Summers Congratulations 7 139
8 Girl Talk Kaylee Summers Once again 8 139


In [17]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
create_user_song_table = "CREATE TABLE IF NOT EXISTS SONGS_USERS"
create_user_song_table = create_user_song_table + "(user_id int,first_name text,last_name text,song_title text, PRIMARY KEY(song_title,user_id))"
try :
    session.execute(create_user_song_table)
except Exception as e:
    print(e)

# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO SONGS_USERS(user_id,first_name,last_name,song_title)"
        query = query + "VALUES(%s,%s,%s,%s)"
    
        session.execute(query, (int(line[10]),line[1],line[4],line[9]))

                    

In [19]:
query = "SELECT song_title,first_name,last_name FROM SONGS_USERS WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name,row.last_name,row.song_title)

Jacqueline Lynch All Hands Against His Own
Tegan Levine All Hands Against His Own
Sara Johnson All Hands Against His Own


### Drop the tables before closing out the sessions

In [20]:
drop_table_song_users = "DROP TABLE IF EXISTS SONGS_USERS"
drop_table_user_session = "DROP TABLE IF EXISTS USER_SESSIONS"
drop_table_music_session = "DROP TABLE IF EXISTS MUSIC_SESSIONS"
try:
    session.execute(drop_table_song_users)
    session.execute(drop_table_user_session)
    session.execute(drop_table_music_session)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()